# Robust Phase Estimation: one and two qubit examples

In [ ]:
import numpy as np
from numpy import pi
from forest_benchmarking import rpe
from pyquil import get_qc
from pyquil.gates import RZ, RX, RY

# get a qauntum computer object. Here we use a noisy qvm. 
qc = get_qc("9q-square", as_qvm=True, noisy=True)

# Below we provide alternative helper functions for more control over noise
# If you want to use this approach to adding noise, start with a noiseless qvm:
# qc = get_qc("9q-generic", as_qvm=True, noisy=False)

# (Helper Functions for Adding Custom Noise to Programs on Noisless QVM)

In [ ]:
# from pyquil.noise import damping_after_dephasing
# from pandas import Series
# from pyquil import Program
# from pyquil.quilbase import Measurement

# def add_damping_dephasing_noise(prog, T1, T2, gate_time):        
#     p = Program()
#     p.defgate("noise", np.eye(2))
#     p.define_noisy_gate("noise", [0] , damping_after_dephasing(T1, T2, gate_time))
#     for elem in prog:
#         p.inst(elem)
#         if isinstance(elem, Measurement):
#             continue # skip measurement
#         p.inst(("noise", 0))
#     return p

# def add_noise_to_experiments(experiments, t1, t2, p00, p11):  
#     gate_time = 200*10**(-9)
#     experiments["Experiment"] = Series([ \
#         add_damping_dephasing_noise(prog, t1, t2, gate_time).define_noisy_readout(0, p00, p11) \
#         for prog in experiments["Experiment"].values])
        
    
# # when running the rpe experiments
# experiments=rpe.generate_rpe_experiments(RZ(angle, 0), num_depths)
# T1 = 20*10**(-6.)
# T2 = 10*10**(-6.)
# add_noise_to_experiments(experiments, T1, T2, .90, .85)
# experiments = rpe.acquire_rpe_data(experiments, qc, multiplicative_factor = factor, additive_error = add_error)


# Generate a Dataframe with Experiments

In [ ]:
# we start with determination of an angle of rotation about the Z axis
num_depths = 6 # max depth of 2^(num_depths - 1)
angle = 2 # radians
rotation = RZ(angle, 0)
experiments=rpe.generate_rpe_experiments(rotation, num_depths)

# Acquire the Data

In [ ]:
experiments = rpe.acquire_rpe_data(experiments, qc)

# Compute the Estimate

In [ ]:
xs, ys, x_stds, y_stds = rpe.find_expectation_values(experiments)
observed = rpe.robust_phase_estimate(xs, ys, x_stds, y_stds)
print("Expected: ", angle)
print("Observed: ", observed)

# Compute a Predicted Upper Bound on the Point Estimate Variance

In [ ]:
var = rpe.get_variance_upper_bound(experiments)
# difference between obs and actual should be less than predicted error 
print(np.abs(observed - angle), ' < ', np.sqrt(var))

# 1q plot of RPE after each iteration

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

angle = pi/16
num_depths = 6 # max depth of 2^(num_depths - 1)
additive_error = .15 # assume noise model with some additive error

experiments = rpe.generate_rpe_experiments(RZ(angle,0), num_depths)
experiments = rpe.acquire_rpe_data(experiments, qc, multiplicative_factor = 100.0, additive_error = additive_error)
xs, ys, x_stds, y_stds = rpe.find_expectation_values(experiments)
observed = rpe.robust_phase_estimate(xs, ys, x_stds, y_stds)
print("Expected: ", angle)
print("Observed: ", observed)

expected = [(1.0, angle*2**j) for j in range(num_depths)]
ax = rpe.plot_RPE_iterations(experiments, expected)
plt.show()

# 1q phase estimation of RZ for various angles and hyperparameters (SLOW)

In [ ]:
angles = pi*np.linspace(0,2.0,3)
num_depths = 5 # max depth of 2^(num_depths - 1)
multiplicative_factors = [1.0, 1000.] # Multiply the optimal number of shots by this factor for each experiment
additive_errors = [None, .15] # Try to correct for this additivie error by increasing the number of shots in an optimal way

#scan over angles and do RPE twice on each angle. The second result should agree more because there are more shots
for angle in angles:
    for add_error in additive_errors:
        print("Assuming additive error of " + str(add_error))
        for factor in multiplicative_factors:
            print("With multiplicative factor x" + str(factor))
            experiments=rpe.generate_rpe_experiments(RZ(angle, 0), num_depths)
            experiments = rpe.acquire_rpe_data(experiments, qc, multiplicative_factor = factor, additive_error = add_error)
            xs, ys, x_stds, y_stds = rpe.find_expectation_values(experiments)
            observed = rpe.robust_phase_estimate(xs, ys, x_stds, y_stds)
            print("Expected: ", angle)
            print("Observed: ", observed)
            print()
    print()

# 1q phase estimation of rotation around 'Hadamard Axis' for various angles (SLOW)

In [ ]:
from pyquil import Program

angles = pi*np.linspace(0,2.0,3)
num_depths = 5 # max depth of 2^(num_depths - 1)
mult_factor = 1.0 # Multiply the optimal number of shots by this factor for each experiment
add_error = None # Try to correct for this additivie error by increasing the number of shots in an optimal way
 
#scan over angles and do RPE twice on each angle. The second result should agree more because there are more shots
for angle in angles:
    RH = Program(RY(-pi/4,0)).inst(RZ(angle,0)).inst(RY(pi/4,0))
    #specify axis by a tuple of (theta, phi) with the usual spherical coordinates
    experiments=rpe.generate_rpe_experiments(RH, num_depths, axis = (pi/4,0))
    experiments = rpe.acquire_rpe_data(experiments, qc, multiplicative_factor = mult_factor, additive_error = add_error)
    xs, ys, x_stds, y_stds = rpe.find_expectation_values(experiments)
    observed = rpe.robust_phase_estimate(xs, ys, x_stds, y_stds)
    print("Expected: ", angle)
    print("Observed: ", observed)
    print()

# Characterizing a universal 1q gateset with approximately orthogonal rotation axes (using simulated artificially imperfect gates)

In [ ]:
from pyquil import Program
from pyquil.quil import DefGate
qc = get_qc("9q-square", as_qvm=True, noisy=True)


"""
Procedure and notation follows Sections III A, B, and C in 
[RPE]  Robust Calibration of a Universal Single-Qubit Gate-Set via Robust Phase Estimation
            Kimmel et al., Phys. Rev. A 92, 062315 (2015)
            https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.062315

"""    
pauli_x = np.array([[0,1],[1,0]])
pauli_z = np.array([[1,0],[0,-1]])

alpha = .01
epsilon = .02
theta = .5

# Section III A of [RPE]
gate1 = RZ(pi/2 * (1 + alpha), 0) # assume some small over rotation alpha

# let gate 2 be RX(pi/4) with over rotation epsilon,
# and with a slight over-tilt of rotation axis by theta in X-Z plane
mtrx = np.add(np.cos(pi/8 * (1 + epsilon)) * np.eye(2),
                - 1j * np.sin(pi/8 * (1 + epsilon)) \
                * (np.add(np.cos(theta) * pauli_x, np.sin(theta) * pauli_z)))
# Section III B of [RPE]

# get Quil definition for simulated imperfect gate
definition = DefGate('ImperfectRX', mtrx)
# get gate constructor
IRX = definition.get_constructor()
# set gate as program with definition and instruction, compiled into native gateset
gate2 = qc.compiler.quil_to_native_quil(Program([definition, IRX(0)]))

# Section III B of [RPE], eq. III.3
# construct the program used to estimate theta
half = Program(gate1)
for _ in range(4):
    half.inst(IRX(0))
half.inst(gate1)
# compile into native gateset 
U_theta =  qc.compiler.quil_to_native_quil(Program([definition, half, half]))

num_depths = 6 # max depth of 2^(num_depths - 1)

results = []
for idx, gate in enumerate([gate1, gate2, U_theta]):
    axis = (pi/2,0) #gate2 and U_theta assumed to be rotations about X axis
    if idx == 0:
        axis = None
    experiments = rpe.generate_rpe_experiments(gate, num_depths, axis=axis)
    experiments = rpe.acquire_rpe_data(experiments, qc, multiplicative_factor = 10.0)
    xs, ys, x_stds, y_stds = rpe.find_expectation_values(experiments)
    result = rpe.robust_phase_estimate(xs, ys, x_stds, y_stds)
    results += [result]
    
print("Expected Alpha: " + str(alpha))
print("Estimated Alpha: " + str(results[0]/(pi/2) - 1))
print()
print("Expected Epsilon: " + str(epsilon))
epsilon_est = results[1]/(pi/4) - 1
print("Estimated Epsilon: " + str(epsilon_est))
print()
print("Expected Theta: " + str(theta))
print("Estimated Theta: " + str(np.sin(results[2]/2)/(2*np.cos(epsilon_est * pi/2))))

# 2q CZ calibration

In [ ]:
from pyquil.gates import CZ
num_depths = 5 # max depth of 2^(num_depths - 1)
additive_error = .15

EXP_TYPES = zip( ['+/0', '0/+', '+/1', '1/+'] , zip([0, 1, 0, 1], [False, False, True, True]))
for label, exp_type in EXP_TYPES: 
    print("Experiment " + label)
    experiments = rpe.generate_rpe_experiments(Program(CZ(0, 1)), num_depths,measurement_qubit = exp_type[0],init_one = exp_type[1])
    
    if (label[0] == '1' or label[2] == '1'):
        print("Expected Angle: " + str(pi))
              
    if (label[0] == '0' or label[2] == '0'):
            print("Expected Angle: " + str(0))

    experiments = rpe.acquire_rpe_data(experiments, qc, multiplicative_factor = 10.0, additive_error = additive_error)
    xs, ys, x_stds, y_stds = rpe.find_expectation_values(experiments)
    result = rpe.robust_phase_estimate(xs, ys, x_stds, y_stds)
    print("Observerd: " + str(result))
    print()
